# RNN text classification

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding

## Import Data

In [2]:
movies = pd.read_csv('./data/movies_cleaned.csv')

In [3]:
# gonna start with a simple binary classification task using the two most common genres
data = movies[movies['Genre'].isin(['comedy', 'drama'])].copy(deep=True)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10343 entries, 1 to 20956
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      10343 non-null  int64 
 1   Title             10343 non-null  object
 2   Origin/Ethnicity  10343 non-null  object
 3   Director          10343 non-null  object
 4   Genre             10343 non-null  object
 5   Plot              10343 non-null  object
 6   plot_length       10343 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 646.4+ KB


## Baseline

In [5]:
data['Genre'].value_counts(normalize=True)

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

drama     0.576622
comedy    0.423378
Name: Genre, dtype: float64

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data['Plot'], data['Genre'])

## Prepare text

In [7]:
tokenizer = Tokenizer(1000)
tokenizer.fit_on_texts(X_train)

In [8]:
X_train_token = tokenizer.texts_to_sequences(X_train)

In [9]:
X_test_token = tokenizer.texts_to_sequences(X_test)

In [10]:
data['plot_length'].max()

2958

In [21]:
X_train_seq = pad_sequences(X_train_token, maxlen = 1000)

In [22]:
X_test_seq = pad_sequences(X_test_token, maxlen = 1000)

In [13]:
y_train_token = np.where((y_train == 'drama'), 1, 0)

In [14]:
y_test_token = np.where((y_test == 'drama'), 1, 0)

In [15]:
y_train_token

array([1, 1, 1, ..., 0, 0, 1])

## Model

### RNN

In [23]:
model = Sequential()
model.add(Embedding(input_dim = tokenizer.num_words, output_dim = 64))
model.add(SimpleRNN(16))
model.add(Dense(200, activation = 'relu'))
model.add(Dense(200, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [24]:
model.compile(loss='bce', optimizer = 'adam', metrics = ['acc'])

In [26]:
history = model.fit(X_train_seq, y_train_token, epochs = 10, validation_data=(X_test_seq, y_test_token))

Epoch 1/10
243/243 [==============================] - 51s 211ms/step - loss: 0.2651 - acc: 0.8975 - val_loss: 0.8740 - val_acc: 0.6210
Epoch 2/10
243/243 [==============================] - 51s 212ms/step - loss: 0.1780 - acc: 0.9318 - val_loss: 1.1480 - val_acc: 0.6500
Epoch 3/10
243/243 [==============================] - 49s 204ms/step - loss: 0.1230 - acc: 0.9538 - val_loss: 1.3839 - val_acc: 0.6419
Epoch 4/10
243/243 [==============================] - 49s 203ms/step - loss: 0.1079 - acc: 0.9624 - val_loss: 1.3265 - val_acc: 0.6350
Epoch 5/10
243/243 [==============================] - 49s 202ms/step - loss: 0.0621 - acc: 0.9783 - val_loss: 1.8383 - val_acc: 0.6458
Epoch 6/10
243/243 [==============================] - 50s 204ms/step - loss: 0.0589 - acc: 0.9813 - val_loss: 1.9567 - val_acc: 0.6199
Epoch 7/10
243/243 [==============================] - 49s 203ms/step - loss: 0.0635 - acc: 0.9774 - val_loss: 1.5670 - val_acc: 0.6299
Epoch 8/10
243/243 [==============================] - 4

### LSTM

In [27]:
model2 = Sequential()
model2.add(Embedding(input_dim = tokenizer.num_words, output_dim = 64))
model2.add(LSTM(16))
model2.add(Dense(200, activation = 'relu'))
model2.add(Dense(200, activation = 'relu'))
model2.add(Dense(1, activation = 'sigmoid'))
model2.compile(loss='bce', optimizer = 'adam', metrics = ['acc'])

In [28]:
history2 = model2.fit(X_train_seq, y_train_token, epochs = 10, validation_data=(X_test_seq, y_test_token))

Epoch 1/10
243/243 [==============================] - 17s 68ms/step - loss: 0.6720 - acc: 0.5937 - val_loss: 0.5772 - val_acc: 0.6821
Epoch 2/10
243/243 [==============================] - 17s 68ms/step - loss: 0.5283 - acc: 0.7491 - val_loss: 0.5761 - val_acc: 0.7258
Epoch 3/10
243/243 [==============================] - 17s 68ms/step - loss: 0.4834 - acc: 0.7746 - val_loss: 0.5613 - val_acc: 0.7258
Epoch 4/10
243/243 [==============================] - 16s 66ms/step - loss: 0.4388 - acc: 0.8085 - val_loss: 0.5643 - val_acc: 0.7231
Epoch 5/10
243/243 [==============================] - 16s 66ms/step - loss: 0.3996 - acc: 0.8235 - val_loss: 0.5984 - val_acc: 0.7320
Epoch 6/10
243/243 [==============================] - 16s 65ms/step - loss: 0.3767 - acc: 0.8317 - val_loss: 0.6605 - val_acc: 0.7162
Epoch 7/10
243/243 [==============================] - 16s 67ms/step - loss: 0.3596 - acc: 0.8464 - val_loss: 0.6426 - val_acc: 0.7038
Epoch 8/10
243/243 [==============================] - 16s 68ms

### GRU

In [30]:
model3 = Sequential()
model3.add(Embedding(input_dim = tokenizer.num_words, output_dim = 64))
model3.add(GRU(16))
model3.add(Dense(200, activation = 'relu'))
model3.add(Dense(200, activation = 'relu'))
model3.add(Dense(1, activation = 'sigmoid'))
model3.compile(loss='bce', optimizer = 'adam', metrics = ['acc'])

In [31]:
history3 = model3.fit(X_train_seq, y_train_token, epochs = 10, validation_data=(X_test_seq, y_test_token))

Epoch 1/10
243/243 [==============================] - 199s 817ms/step - loss: 0.6768 - acc: 0.5714 - val_loss: 0.6544 - val_acc: 0.6516
Epoch 2/10
  6/243 [..............................] - ETA: 3:10 - loss: 0.6622 - acc: 0.6447

KeyboardInterrupt: 

## Gonna optimize my LSTM model

In [32]:
X_train_seq2 = pad_sequences(X_train_token, maxlen = data['plot_length'].max())
X_test_seq2 = pad_sequences(X_test_token, maxlen = data['plot_length'].max())

In [35]:
from tensorflow.keras.layers import Dropout

In [36]:
model4 = Sequential()
model4.add(Embedding(input_dim = tokenizer.num_words, output_dim = 64))
model4.add(LSTM(16))
model4.add(Dense(200, activation = 'relu'))
model4.add(Dropout(0.2))
model4.add(Dense(200, activation = 'relu'))
model4.add(Dropout(0.2))
model4.add(Dense(1, activation = 'sigmoid'))
model4.compile(loss='bce', optimizer = 'adam', metrics = ['acc'])

In [37]:
history4 = model4.fit(X_train_seq2, y_train_token, epochs = 10, validation_data=(X_test_seq2, y_test_token))

Epoch 1/10
243/243 [==============================] - 49s 201ms/step - loss: 0.6670 - acc: 0.6013 - val_loss: 0.6714 - val_acc: 0.6160
Epoch 2/10
243/243 [==============================] - 48s 200ms/step - loss: 0.5245 - acc: 0.7481 - val_loss: 0.7063 - val_acc: 0.5882
Epoch 3/10
243/243 [==============================] - 49s 202ms/step - loss: 0.4853 - acc: 0.7800 - val_loss: 0.7003 - val_acc: 0.6222
Epoch 4/10
243/243 [==============================] - 48s 198ms/step - loss: 0.4523 - acc: 0.7907 - val_loss: 0.8374 - val_acc: 0.6179
Epoch 5/10
243/243 [==============================] - 49s 201ms/step - loss: 0.4137 - acc: 0.8137 - val_loss: 0.7638 - val_acc: 0.5994
Epoch 6/10
243/243 [==============================] - 48s 199ms/step - loss: 0.4520 - acc: 0.8020 - val_loss: 0.9622 - val_acc: 0.5916
Epoch 7/10
243/243 [==============================] - 49s 202ms/step - loss: 0.3861 - acc: 0.8381 - val_loss: 1.0345 - val_acc: 0.6009
Epoch 8/10
243/243 [==============================] - 4